In [1]:
import os
import json

### Helper functions

In [2]:
def clean_list(alist):
    if ".DS_Store" in alist:
        alist.remove(".DS_Store")
    return alist

def save_review(review_file, review_list, reviews_dir):
    file_path = os.path.join(reviews_dir, review_file)
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(review_list, f, ensure_ascii=False, indent=2)

def load_review_files(reviews_dir):
    paper_reviews = clean_list(os.listdir(reviews_dir))
    return paper_reviews

def load_review(reviews_dir, review_file):
    file_path = os.path.join(reviews_dir, review_file)
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)


In [3]:
def get_processed_review(full_review):
    review_content = []
    
    for thread in full_review.values():
        review = thread['review']["content"]
        replies = thread['replies']

        # Extract review fields
        summary = review.get("summary", {}).get("value", "")
        strengths = review.get("strengths", {}).get("value", "")
        weaknesses = review.get("weaknesses", {}).get("value", "")
        questions = review.get("questions", {}).get("value", "")
        ethics = review.get("flag_for_ethics_review", {}).get("value", "")

        # Build the formatted review string
        formatted_review = f"""#### Review:

Summary:
{summary}

Strengths:
{strengths}

Weaknesses:
{weaknesses}

Questions:
{questions}

Ethics:
{ethics}

#### Replies
"""

        # Append replies
        for i, reply in enumerate(replies):
            reply_text = reply["content"]["comment"]["value"]
            formatted_review += f"\n **Reply # {i+1}**\n{reply_text}\n"

        # Add the full formatted string to the list
        review_content.append(formatted_review.strip())

    return review_content

### Run Code

In [4]:
def process_papers(review_files, raw_reviews_dir, processed_reviews_dir):
    for review_file in review_files:
        try:
            review = load_review(raw_reviews_dir, review_file)
            processed_reviews = get_processed_review(review)
            save_review(review_file, processed_reviews, processed_reviews_dir)
        except Exception as e:
            print(e)

In [5]:
accepted_review_files = load_review_files("ICLR2025_Reviews_Raw/accepted")
rejected_review_files = load_review_files("ICLR2025_Reviews_Raw/rejected")

process_papers(accepted_review_files, "ICLR2025_Reviews_Raw/accepted", "ICLR2025_Reviews_Processed/accepted")
process_papers(rejected_review_files, "ICLR2025_Reviews_Raw/rejected", "ICLR2025_Reviews_Processed/rejected")